In [1]:
import ROOT
import numpy as np
import pandas as pd 
import json
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter

ROOT.gROOT.ProcessLine( "gErrorIgnoreLevel = kError;");
ROOT.gStyle.SetCanvasDefW(2400);
ROOT.gStyle.SetCanvasDefH(900);
ROOT.gStyle.SetLegendTextSize(0.05)
ROOT.gStyle.SetLabelSize(0.05)
ROOT.gStyle.SetMarkerSize(1)
ROOT.gStyle.SetMarkerStyle(8)
ROOT.gStyle.SetLineWidth(2)
ROOT.gStyle.SetTickLength(0.02, "y")

Welcome to JupyROOT 6.31/01


In [2]:
def convert_to_type(df, col, type):
    df[col] =  df[col].apply(lambda s: type(s))

def normalize_df(df, cols, norm):
    for col in cols:
        df[col] = df[col].div(df[norm])

In [10]:
df = pd.read_csv("das6-gpu/20231206-174647")
df["nvals"] =  df["input"].apply(lambda s: np.float64(s.split("_")[-1].split(".")[0]))
df["distribution"] =  df["input"].apply(lambda s: s.split("_")[1])
convert_to_type(df, "edges", np.bool_)
convert_to_type(df, "ttotal", np.float64)
df["type"] = "Buffers"

df_usm = pd.read_csv("das6-gpu/20231211-155923")
df_usm["nvals"] =  df_usm["input"].apply(lambda s: np.float64(s.split("_")[-1].split(".")[0]))
df_usm["distribution"] =  df_usm["input"].apply(lambda s: s.split("_")[1])
convert_to_type(df_usm, "edges", np.bool_)
convert_to_type(df_usm, "ttotal", np.float64)
df_usm["type"] = "USM"

df = pd.concat([df, df_usm])
del df["input"]
df

,iter,env,gpu,nbins,bulksize,edges,tfindbin,tfill,tstats,ttotal,nvals,distribution,type
0,0,DPC++,A4000,1,32768,False,0.0,0.0,0.0,3.158348,5.000000e+07,uniform,Buffers
1,0,DPC++,A4000,1,32768,True,0.0,0.0,0.0,3.143772,5.000000e+07,uniform,Buffers
2,0,DPC++,A4000,1,32768,False,0.0,0.0,0.0,3.593378,1.000000e+08,uniform,Buffers
3,0,DPC++,A4000,1,32768,True,0.0,0.0,0.0,4.094069,1.000000e+08,uniform,Buffers
4,0,DPC++,A4000,1,32768,False,0.0,0.0,0.0,9.966019,5.000000e+08,uniform,Buffers
...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,4,DPC++,A4000,1000,32768,True,0.0,0.0,0.0,3.748883,1.000000e+08,uniform,USM
316,4,DPC++,A4000,1000,32768,False,0.0,0.0,0.0,10.399123,5.000000e+08,uniform,USM
317,4,DPC++,A4000,1000,32768,True,0.0,0.0,0.0,10.190794,5.000000e+08,uniform,USM
318,4,DPC++,A4000,1000,32768,False,0.0,0.0,0.0,18.280992,1.000000e+09,uniform,USM


In [12]:
gp = df.groupby(["env", "gpu", "type", "distribution", "nvals", "nbins", "bulksize", "edges"])
gp.mean()

iter  \
env         gpu   type    distribution nvals        nbins bulksize edges         
AdaptiveCpp A4000 Buffers uniform      5.000000e+07 1     32768    False   2.0   
                                                                   True    2.0   
                                                    10    32768    False   2.0   
                                                                   True    2.0   
                                                    100   32768    False   2.0   
...                                                                        ...   
DPC++       A4000 USM     uniform      1.000000e+09 10    32768    True    2.0   
                                                    100   32768    False   2.0   
                                                                   True    2.0   
                                                    1000  32768    False   2.0   
                                                                   True    2.0   

                                                                          tfindbin  \
env         gpu   type    distribution nvals        nbins bulksize edges             
AdaptiveCpp A4000 Buffers uniform      5.000000e+07 1     32768    False       0.0   
                                                                   True        0.0   
                                                    10    32768    False       0.0   
                                                                   True        0.0   
                                                    100   32768    False       0.0   
...                                                                            ...   
DPC++       A4000 USM     uniform      1.000000e+09 10    32768    True        0.0   
                                                    100   32768    False       0.0   
                                                                   True        0.0   
                                                    1000  32768    False       0.0   
                                                                   True        0.0   

                                                                          tfill  \
env         gpu   type    distribution nvals        nbins bulksize edges          
AdaptiveCpp A4000 Buffers uniform      5.000000e+07 1     32768    False    0.0   
                                                                   True     0.0   
                                                    10    32768    False    0.0   
                                                                   True     0.0   
                                                    100   32768    False    0.0   
...                                                                         ...   
DPC++       A4000 USM     uniform      1.000000e+09 10    32768    True     0.0   
                                                    100   32768    False    0.0   
                                                                   True     0.0   
                                                    1000  32768    False    0.0   
                                                                   True     0.0   

                                                                          tstats  \
env         gpu   type    distribution nvals        nbins bulksize edges           
AdaptiveCpp A4000 Buffers uniform      5.000000e+07 1     32768    False     0.0   
                                                                   True      0.0   
                                                    10    32768    False     0.0   
                                                                   True      0.0   
                                                    100   32768    False     0.0   
...                                                                          ...   
DPC++       A4000 USM     uniform      1.000000e+09 10    32768    True      0.0   
                                                    

In [15]:
unique_env = df["env"].unique()
unique_bulksize = df["bulksize"].unique()
unique_nbins = df["nbins"].unique()
unique_nvals = df["nvals"].unique()
unique_gpus = df["gpu"].unique()
unique_distributions = df["distribution"].unique()
unique_edges = [True, False]
unique_type = ["Buffers", "USM"]
unique_env, unique_gpus, unique_type, unique_nbins, unique_bulksize, unique_edges, unique_nvals, unique_distributions

(array(['DPC++', 'AdaptiveCpp', 'CUDA_HIST'], dtype=object),
 array(['A4000'], dtype=object),
 ['Buffers', 'USM'],
 array([   1,   10,  100, 1000]),
 array([32768]),
 [True, False],
 array([5.e+07, 1.e+08, 5.e+08, 1.e+09]),
 array(['uniform'], dtype=object))

## A4000

In [24]:
selected_env = unique_env
selected_bulksize = 32768
selected_type = unique_type
selected_bins = 1000
selected_nvals = unique_nvals
selected_gpu = "A4000"
selected_distr = "uniform"
selected_edges = True

In [75]:
%jsroot on
title = f"Total Runtime of Histo1D with Different Input Sizes on NVIDIA A4000"

w = 800
h = 450
c = ROOT.TCanvas("c1", "", w, h)
c.SetRightMargin(0.39)
c.SetBottomMargin(0.3)

mg = ROOT.TMultiGraph()

l = ROOT.TLegend(0.62, 0.533, 1, 0.9)
l.SetTextSize(0.05)
markerstyles = [21, 25, 22, 26, 20]

ROOT.gStyle.SetPalette(ROOT.kRainbow)  
for ei, env in enumerate(selected_env):
    for ti, type in enumerate(selected_type):
        if type == "USM" and "CUDA" in env:
            continue 
        avg = np.array(gp.mean().loc[env, selected_gpu, type, selected_distr, :, 
                       selected_bins, selected_bulksize, selected_edges]["ttotal"], dtype=np.float64)
        std = np.array(gp.std().loc[env, selected_gpu, type, selected_distr, :,
                       selected_bins, selected_bulksize, selected_edges]["ttotal"], dtype=np.float64)
        gr = ROOT.TGraphErrors(len(selected_nvals), selected_nvals.astype(np.float64), avg, 
                               np.repeat(0., len(selected_nvals)), std)
    
        color = ROOT.TColor.GetPalette()[50+ 60 * ei]
        gr.SetMarkerColor(color)
        gr.SetLineColor(color)
        gr.SetLineWidth(2)
        gr.SetMarkerSize(2)
        gr.SetMarkerStyle(markerstyles[2*ei + ti])
        
        gr.GetHistogram().SetMinimum(0)
        gr.GetHistogram().SetLineWidth(10)
    
        mg.Add(gr, "ALP ")
        l.AddEntry(gr, env.split("_")[0] + " " + type)

# mg.SetTitle(title)

xaxis = mg.GetXaxis()
# xaxis.SetTitle("#splitline{Bulk Size}")
xaxis.SetTitle("Number of values")
# xaxis.SetRangeUser(0, 10000000)
# xaxis.SetTitleOffset(1.5)
xaxis.SetTitleSize(0.05)
xaxis.SetLabelSize(0.05)
# xaxis.LabelsOption("hM")
# xaxis.SetTickSize(0)

yaxis = mg.GetYaxis()
yaxis.SetTitle("Time (s)")
yaxis.SetTitleOffset(1)
yaxis.SetTitleSize(0.05)
yaxis.SetLabelSize(0.05)

ROOT.gStyle.SetTitleFontSize(0.1)

mg.Draw("a")
l.Draw()
c.DrawClone()
# c.SaveAs(f"nsys_histogram_{env}.png")

## A6000

## A2